### 골빈해커의 3분 딥러닝
#### 10장 번역과 챗봇 모델의 기본 RNN

#### 단어자동완성

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u', 
            'v', 'w', 'x', 'y', 'z']

# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k': 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']


In [3]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [4]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

In [5]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype = tf.float32)

outputs = tf.transpose(outputs, [1, 0 ,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

    
print('최적화 완료!')

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값: ', [w[:3] + ' ' for w in seq_data])
print('예측값: ', predict_words)
print('정확도: ', accuracy_val)

Epoch: 0001 cost = 2.697461
Epoch: 0002 cost = 2.025894
Epoch: 0003 cost = 1.409234
Epoch: 0004 cost = 1.142312
Epoch: 0005 cost = 0.478856
Epoch: 0006 cost = 0.564950
Epoch: 0007 cost = 0.429269
Epoch: 0008 cost = 0.358164
Epoch: 0009 cost = 0.267239
Epoch: 0010 cost = 0.277022
Epoch: 0011 cost = 0.273360
Epoch: 0012 cost = 0.348875
Epoch: 0013 cost = 0.363346
Epoch: 0014 cost = 0.300657
Epoch: 0015 cost = 0.385379
Epoch: 0016 cost = 0.179756
Epoch: 0017 cost = 0.065269
Epoch: 0018 cost = 0.076488
Epoch: 0019 cost = 0.115200
Epoch: 0020 cost = 0.110858
Epoch: 0021 cost = 0.040254
Epoch: 0022 cost = 0.115897
Epoch: 0023 cost = 0.036113
Epoch: 0024 cost = 0.030051
Epoch: 0025 cost = 0.024180
Epoch: 0026 cost = 0.059713
Epoch: 0027 cost = 0.012890
Epoch: 0028 cost = 0.002502
Epoch: 0029 cost = 0.024716
Epoch: 0030 cost = 0.012242
최적화 완료!

=== 예측 결과 ===
입력값:  ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값:  ['word', 'wood', 'deep', 'dive', 'cold', 'co